In [2]:
import os
import xarray as xr
import numpy as np

import sys
sys.path.append("..")
import utils

In [3]:
file = '../../data/satellite/sst_rep_l4/sst_1982_2021.nc'
ds = xr.open_mfdataset(file)

In [9]:
# Compute the area of each grid cell using the provided function
grid_areas = utils.area(ds['lat'], ds['lon'])

# Calculate the weighted mean of 'analysed_sst' along the lat and lon dimensions
sst_mean = ds['analysed_sst'].weighted(grid_areas).mean(dim=['lat', 'lon']) - 273.15

# Calculate the weighted mean of 'analysed_sst' + 'analysis_error'
sst_plus_error = (ds['analysed_sst'] + ds['analysis_error']).weighted(grid_areas).mean(dim=['lat', 'lon']) - 273.15

# Calculate the weighted mean of 'analysed_sst' - 'analysis_error'
sst_minus_error = (ds['analysed_sst'] - ds['analysis_error']).weighted(grid_areas).mean(dim=['lat', 'lon']) - 273.15



In [14]:
# Create the new dataset with the computed variables
ds_ts = xr.Dataset({
    'sst': sst_mean,
    'sst_plus_error': sst_plus_error,
    'sst_minus_error': sst_minus_error
})
ds_ts = ds_ts.assign_coords(time=ds['time'])

fold = '../../results/SST/satellite/'
if not os.path.exists(fold):
    os.makedirs(fold)

ds_ts = ds_ts.groupby("time.year").mean(dim="time")
ds_ts.to_netcdf(fold + 'mean_sst_satellite.nc')
